In [6]:
import json
import os
import cv2
import matplotlib.pyplot as plt

# json 데이터의 categories, images, annotations 준비하기

json_path = "./data/instances_default.json"

with open(json_path, 'r', encoding='utf-8') as j:
    json_data = json.load(j)

categories_info = json_data['categories']
images_info = json_data['images']
annotations_info = json_data['annotations']

# label 매칭해줄 dictionary 준비하기

label_dict = {1: 0, 2: 1}

# 모든 image들에 대해

for image_json in images_info:
    # image 준비하기, file_name_temp
    
    file_name = image_json['file_name']
    file_name_temp = file_name.replace(".jpg", "")
    image_path = os.path.join("./data/json_image/", file_name)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # image의 shape 
    
    image_height, image_width, image_channel = image.shape
    
    # image의 ID 준비하기
    
    image_id = image_json['id']
    
    # 모든 annotation들에 대해
    
    for anno_json in annotations_info:
        if image_id == anno_json['image_id']:
            # 이 annotation이 현재 image에 속해있는 게 맞다면
            # bounding box의 좌표 정보 가져오기, yolo 좌표 형태로 변환
            
            bbox = anno_json['bbox']
            
            x = int(bbox[0])
            y = int(bbox[1])
            w = int(bbox[2])
            h = int(bbox[3])
            
            yolo_xcenter = (2*x + w) / (2 * image_width)
            yolo_ycenter = (2*y + h) / (2 * image_height)
            yolo_width = w / image_width
            yolo_height = h / image_height
            
            # label 정보 가져오기
            
            category_id = anno_json['category_id']
            label_number = label_dict[category_id]
            
            # txt 파일로 저장
            
            with open(f"{file_name_temp}.txt", "a") as f:
                f.write(f"{label_number} {yolo_xcenter} {yolo_ycenter} {yolo_width} {yolo_height}\n")